In [1]:
from tira.rest_api_client import Client
import pandas as pd
from tqdm import tqdm
import ir_datasets
import json
from pathlib import Path
from tira.third_party_integrations import ensure_pyterrier_is_loaded, load_rerank_data, normalize_run

datasets = {
    'longeval/heldout': 'longeval-heldout-20230513-training',
    'longeval/b-long-september': 'longeval-long-september-20230513-training',
    'longeval/a-short-july': 'longeval-short-july-20230513-training',
    'longeval/train': 'longeval-train-20230513-training'
}

import pyterrier as pt

if not pt.started():
    # needed in the image so that pyterrier can find the shared libs
    !ln -s /usr/lib/jvm/java-11-openjdk/lib/server/libjvm.so /usr/lib/jvm/java-11-openjdk/lib/
    pt.init()

def rerank(rerank_data, wmodel):
    output_directory = Path(rerank_data) / 'plain-runs'
    output_file =  output_directory / f'{wmodel}-run.txt.gz'
    print(f'Use output {output_file}')
    if output_file.exists():
        print('Already exists')
        return
    
    print(f'Load Data from {rerank_data} to re-rank with {wmodel}.')
    rerank_data = load_rerank_data(rerank_data)
    print('Re-rank Data')
    pipeline = pt.text.scorer(wmodel=wmodel, verbose=True, body_attr='text')
    
    rerank_data['query'] = rerank_data['query'].apply(lambda i: "".join([x if x.isalnum() else " " for x in i]))

    result = pipeline(rerank_data)

    print(f'writing run file to:\t{output_file}')
    Path(output_directory).mkdir(parents=True, exist_ok=True)
    pt.io.write_results(normalize_run(result, 1000), output_file, run_name=wmodel)


ln: target '/usr/lib/jvm/java-11-openjdk/lib/' is not a directory: No such file or directory


PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [2]:
rerank('longeval-heldout-20230513-training', 'BM25')

Use output longeval-heldout-20230513-training/plain-runs/BM25-run.txt.gz
Already exists


In [5]:
!rm -Rf longeval-heldout-20230513-training/plain-runs